In [12]:
!pip install tqdm
!pip install opencage
!pip install scikit-learn

In [4]:
import pandas as pd
import time
from geopy.geocoders import Nominatim
from tqdm.notebook import tqdm
from opencage.geocoder import OpenCageGeocode
from google.colab import userdata
import requests
import os
from collections import defaultdict

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
csv_path = '/content/drive/MyDrive/ML_Dataset/AQI_2010_2023/stations_info.csv'
df = pd.read_csv(csv_path)
df.head()

,file_name,state,city,agency,station_location,start_month,start_month_num,start_year
0,AP001,Andhra Pradesh,Tirupati,APPCB,"Tirumala, Tirupati",July,7,2016
1,AP002,Andhra Pradesh,Vijayawada,APPCB,"PWD Grounds, Vijayawada",May,5,2017
2,AP003,Andhra Pradesh,Visakhapatnam,APPCB,"GVM Corporation, Visakhapatnam",July,7,2017
3,AP004,Andhra Pradesh,Rajamahendravaram,APPCB,"Anand Kala Kshetram, Rajamahendravaram",September,9,2017
4,AP005,Andhra Pradesh,Amaravati,APPCB,"Secretariat, Amaravati",November,11,2017


In [ ]:
df['station_address'] = df['station_location'] + ', ' +df['state']
df.head()

,file_name,state,city,agency,station_location,start_month,start_month_num,start_year,station_address
0,AP001,Andhra Pradesh,Tirupati,APPCB,"Tirumala, Tirupati",July,7,2016,"Tirumala, Tirupati , Andhra Pradesh"
1,AP002,Andhra Pradesh,Vijayawada,APPCB,"PWD Grounds, Vijayawada",May,5,2017,"PWD Grounds, Vijayawada , Andhra Pradesh"
2,AP003,Andhra Pradesh,Visakhapatnam,APPCB,"GVM Corporation, Visakhapatnam",July,7,2017,"GVM Corporation, Visakhapatnam , Andhra Pradesh"
3,AP004,Andhra Pradesh,Rajamahendravaram,APPCB,"Anand Kala Kshetram, Rajamahendravaram",September,9,2017,"Anand Kala Kshetram, Rajamahendravaram , Andhr..."
4,AP005,Andhra Pradesh,Amaravati,APPCB,"Secretariat, Amaravati",November,11,2017,"Secretariat, Amaravati , Andhra Pradesh"


In [ ]:
def compute_latitude_longitude(row):
  try:
    location = geolocator.geocode(row['station_address'])
    time.sleep(3)
    if location:
        return pd.Series({'latitude': location.latitude, 'longitude': location.longitude})
    else:
        return pd.Series({'latitude': None, 'longitude': None})
  except:
    return pd.Series({'latitude': None, 'longitude': None})

In [ ]:
geolocator = Nominatim(user_agent="pollution_station_locator")
tqdm.pandas()
df[['latitude', 'longitude']] = df.progress_apply(compute_latitude_longitude, axis=1)

  0%|          | 0/453 [00:00<?, ?it/s]

In [ ]:
df.to_csv('/content/drive/MyDrive/ML_Dataset/AQI_2010_2023_updated/stations_with_lat_lon.csv', index=False)

Loading intermediate data for latitude and longitude

In [ ]:
csv_path = '/content/drive/MyDrive/ML_Dataset/AQI_2010_2023_updated/stations_with_lat_lon.csv'
df = pd.read_csv(csv_path)
retry_df = df[df['latitude'].isna()]
retry_df.head()

,file_name,state,city,agency,station_location,start_month,start_month_num,start_year,station_address,latitude,longitude
1,AP002,Andhra Pradesh,Vijayawada,APPCB,"PWD Grounds, Vijayawada",May,5,2017,"PWD Grounds, Vijayawada , Andhra Pradesh",NaN,NaN
2,AP003,Andhra Pradesh,Visakhapatnam,APPCB,"GVM Corporation, Visakhapatnam",July,7,2017,"GVM Corporation, Visakhapatnam , Andhra Pradesh",NaN,NaN
3,AP004,Andhra Pradesh,Rajamahendravaram,APPCB,"Anand Kala Kshetram, Rajamahendravaram",September,9,2017,"Anand Kala Kshetram, Rajamahendravaram , Andhr...",NaN,NaN
7,AP008,Andhra Pradesh,Tirupati,APPCB,"Vaikuntapuram, Tirupati",November,11,2022,"Vaikuntapuram, Tirupati , Andhra Pradesh",NaN,NaN
9,AP010,Andhra Pradesh,Kadapa,APPCB,"Yerramukkapalli, Kadapa",January,1,2023,"Yerramukkapalli, Kadapa , Andhra Pradesh",NaN,NaN


Alternate method to get latitude and longitude

In [ ]:
geocoder = OpenCageGeocode(userdata.get('OPENCAGE_API_KEY'))
def compute_latitude_longitude(row):
  try:
    query = row['station_address'] + ", India"
    result = geocoder.geocode(query)
    time.sleep(3)
    if result and len(result):
        return pd.Series({'latitude': result[0]['geometry']['lat'], 'longitude': result[0]['geometry']['lng']})
    else:
        return pd.Series({'latitude': None, 'longitude': None})
  except:
    return pd.Series({'latitude': None, 'longitude': None})

In [ ]:
tqdm.pandas()
retry_df[['latitude', 'longitude']] = retry_df.progress_apply(compute_latitude_longitude, axis=1)

  0%|          | 0/230 [00:00<?, ?it/s]

/tmp/ipython-input-8-3659275104.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retry_df[['latitude', 'longitude']] = retry_df.progress_apply(compute_latitude_longitude, axis=1)


In [ ]:
retry_df

,file_name,state,city,agency,station_location,start_month,start_month_num,start_year,station_address,latitude,longitude
1,AP002,Andhra Pradesh,Vijayawada,APPCB,"PWD Grounds, Vijayawada",May,5,2017,"PWD Grounds, Vijayawada , Andhra Pradesh",16.51928,80.63049
2,AP003,Andhra Pradesh,Visakhapatnam,APPCB,"GVM Corporation, Visakhapatnam",July,7,2017,"GVM Corporation, Visakhapatnam , Andhra Pradesh",17.68009,83.20161
3,AP004,Andhra Pradesh,Rajamahendravaram,APPCB,"Anand Kala Kshetram, Rajamahendravaram",September,9,2017,"Anand Kala Kshetram, Rajamahendravaram , Andhr...",17.00517,81.77784
7,AP008,Andhra Pradesh,Tirupati,APPCB,"Vaikuntapuram, Tirupati",November,11,2022,"Vaikuntapuram, Tirupati , Andhra Pradesh",13.63551,79.41989
9,AP010,Andhra Pradesh,Kadapa,APPCB,"Yerramukkapalli, Kadapa",January,1,2023,"Yerramukkapalli, Kadapa , Andhra Pradesh",14.47750,78.82353
...,...,...,...,...,...,...,...,...,...,...,...
441,WB003,West Bengal,Howrah,WBPCB,"Ghusuri, Howrah",June,6,2016,"Ghusuri, Howrah , West Bengal",22.61667,88.35000
442,WB004,West Bengal,Durgapur,WBPCB,"Sidhu Kanhu Indoor Stadium, Durgapur",July,7,2016,"Sidhu Kanhu Indoor Stadium, Durgapur , West Be...",23.49957,87.32155
443,WB005,West Bengal,Kolkata,WBPCB,"Rabindra Bharati University, Kolkata",September,9,2016,"Rabindra Bharati University, Kolkata , West Be...",22.56263,88.36304
445,WB007,West Bengal,Asansol,WBPCB,"Asansol Court Area, Asansol",February,2,2018,"Asansol Court Area, Asansol , West Bengal",23.68333,86.98333


In [ ]:
merged_df = pd.concat([df.dropna(subset=['latitude', 'longitude']), retry_df]).drop_duplicates(subset=['file_name'])
merged_df = merged_df.sort_values(by='file_name')
merged_df.to_csv('/content/drive/MyDrive/ML_Dataset/AQI_2010_2023_updated/stations_with_lat_lon_final.csv', index=False)

Adding elevation information

In [ ]:
def compute_elevation(row):
  try:
    latitude = row['latitude']
    longitude = row['longitude']
    url = f"https://api.open-elevation.com/api/v1/lookup?locations={latitude},{longitude}"
    response = requests.get(url).json()
    time.sleep(3)
    if response:
        return response['results'][0]['elevation']
    else:
        return None
  except:
    return None

In [ ]:
tqdm.pandas()
merged_df['elevation'] = merged_df.progress_apply(compute_elevation, axis=1)

  0%|          | 0/453 [00:00<?, ?it/s]

In [ ]:
merged_df.to_csv('/content/drive/MyDrive/ML_Dataset/AQI_2010_2023_updated/stations_with_lat_lon_ele.csv', index=False)

Clean and consolidating individual AQI files

In [6]:
folder_path = "/content/drive/MyDrive/ML_Dataset/AQI_2010_2023"
csv_files = [f for f in os.listdir(folder_path) if f.endswith(".csv") and not f.endswith('stations_info.csv')]
file_headers = defaultdict(set)
file_header_counter = {}
for file in tqdm(csv_files):
    file_path = os.path.join(folder_path, file)
    try:
        df = pd.read_csv(file_path, nrows=0)
        for column in df.columns:
              file_headers[column].add(file)
    except Exception as e:
        print(f"Error reading {file}: {e}")
file_headers = {k: sorted(list(v)) for k, v in file_headers.items()}
hedaer_df = pd.DataFrame([
    {"header": header, "files": files, "file_count": len(files)}
    for header, files in file_headers.items()
])

  0%|          | 0/453 [00:00<?, ?it/s]

In [11]:
hedaer_df['header'].values.tolist()
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
import pandas as pd

# Example list of strings
strings = hedaer_df['header'].values.tolist()

# Step 1: TF-IDF Vectorization
vectorizer = TfidfVectorizer(analyzer='char_wb', ngram_range=(2, 4))  # char-level n-grams
X = vectorizer.fit_transform(strings)

# Step 2: Compute similarity matrix
cosine_sim = cosine_similarity(X)

# Step 3: Clustering (80% similarity => 0.2 distance)
threshold = 1 - 0.8
clustering = AgglomerativeClustering(
    affinity='precomputed',
    linkage='complete',
    distance_threshold=threshold,
    n_clusters=None
)
labels = clustering.fit_predict(1 - cosine_sim)

# Step 4: Group by labels
clusters = {}
for idx, label in enumerate(labels):
    clusters.setdefault(label, []).append(strings[idx])

# Display result
for cluster_id, items in clusters.items():
    print(f"Cluster {cluster_id + 1}: {items}")


['From Date',
 'To Date',
 'PM2.5 (ug/m3)',
 'PM10 (ug/m3)',
 'NO (ug/m3)',
 'NO2 (ug/m3)',
 'NOx (ppb)',
 'NH3 (ug/m3)',
 'SO2 (ug/m3)',
 'CO (mg/m3)',
 'Ozone (ug/m3)',
 'Benzene (ug/m3)',
 'Toluene (ug/m3)',
 'Temp (degree C)',
 'RH (%)',
 'WS (m/s)',
 'WD (deg)',
 'SR (W/mt2)',
 'BP (mmHg)',
 'VWS (m/s)',
 'Xylene (ug/m3)',
 'RF (mm)',
 'AT (degree C)',
 'Eth-Benzene (ug/m3)',
 'MP-Xylene (ug/m3)',
 'O Xylene (ug/m3)',
 'VWS (degree)',
 'WD (degree)',
 'Temp ()',
 'RF ()',
 'NO (mg/m3)',
 'AT ()',
 'RF (m/s)',
 'CO (mg/Nm3)',
 'SR (ug/m3)',
 'MP-Xylene ()',
 'CH4 (ug/m3)',
 'NMHC (ug/m3)',
 'THC (ug/m3)',
 'Gust (km/hr)',
 'Variance (n)',
 'Power (W)',
 'CO2 (mg/m3)',
 'Ozone (ppb)',
 'Gust (kl/h)',
 'Gust (m/s)',
 'WS ()',
 'SPM (ug/m3)',
 'MH (m)',
 'HCHO (ug/m3)',
 'Hg (ug/m3)',
 'CO (ug/m3)',
 'Toluene ()',
 'NOx (ug/m3)',
 'WD (degree C)',
 'CO (ng/m3)',
 'Benzene ()',
 'Temp (ug/m3)',
 'SR ()',
 'WD ()',
 'Eth-Benzene ()',
 'Xylene ()',
 'RH ()',
 'BP ()',
 'SO2 ()',
 'CH4 ()